In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import numpy as np

In [2]:
df = pd.read_csv('/Users/glebovmaksim/Python/Nornikel/df_hack_final.csv')
test = pd.read_csv('/Users/glebovmaksim/Python/Nornikel/test.csv')

In [3]:
# Создаём булевый массив, где находим изменения с 0 на 1
change_indices = (df['FM_1.1_A'] == 1) & (df['FM_1.1_A'].shift(1) == 0)

# Выбираем строки, где произошло изменение
result_df = df[change_indices | change_indices.shift(-1).fillna(False)]

/var/folders/5q/2qkk36kx4z330q26mbwy63fr0000gn/T/ipykernel_42341/2884941663.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result_df = df[change_indices | change_indices.shift(-1).fillna(False)]


In [4]:
result_df

,MEAS_DT,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Mass_2,Dens_4,Mass_4,Vol_4,Cu_4F,...,Cu_3.1T_max,Cu_3.1T_min,FM_3.2_A,Cu_3.2C_max,Cu_3.2C_min,Ni_3.2C_max,Ni_3.2C_min,Cu_3.2T_max,Cu_3.2T_min,Ni_rec
826,2024-01-09 14:30:00,3.1820,1.6008,997.5,1236.409180,807.101440,1.398412,883.365784,1651.28125,0.4661,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.00,0.936218
827,2024-01-09 14:45:00,3.1247,1.5917,1082.0,1217.708496,808.573181,1.391624,882.508301,1677.12500,0.4729,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.00,0.970785
990,2024-01-11 07:30:00,3.2381,1.8850,1242.0,1475.192383,769.592163,1.376338,814.016724,1606.43750,0.4625,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.00,0.978325
991,2024-01-11 07:45:00,3.2081,1.9100,1372.0,1435.682373,784.863159,1.381591,824.985291,1612.34375,0.4590,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.00,0.946864
1486,2024-01-16 11:30:00,2.5659,1.6729,849.5,1275.464722,820.953308,1.404230,857.366821,1576.68750,0.5484,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.00,0.967483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29057,2024-10-29 16:15:00,2.0294,1.5748,1052.0,1323.386108,745.476562,1.364443,745.126892,1521.65625,0.2715,...,1.8,0.8,0.0,16.5,13.0,3.7,3.5,1.8,0.75,0.946520
29584,2024-11-04 04:00:00,2.9648,1.3298,1008.5,1249.110474,674.456360,1.385611,791.776245,1530.84375,0.3502,...,1.8,0.8,0.0,16.5,13.0,3.7,3.5,1.8,0.75,0.914733
29585,2024-11-04 04:15:00,2.9747,1.3254,1124.0,1261.323853,665.971619,1.385929,795.058472,1533.43750,0.3898,...,1.8,0.8,0.0,16.5,13.0,3.7,3.5,1.8,0.75,0.967756
29896,2024-11-07 10:00:00,3.0917,1.4658,886.0,1333.236572,779.978027,1.346435,722.516357,1555.87500,0.4919,...,1.8,0.8,0.0,16.5,13.0,3.7,3.5,1.8,0.75,0.937388


In [5]:
BIG_result_df = pd.DataFrame()

In [6]:
#1

In [7]:
for i in range(1, 3):

        gold_ranges = [f'Ni_1.{i}C_min', f'Ni_1.{i}C_max', f'Cu_1.{i}C_min', f'Cu_1.{i}C_max']

        X = result_df[result_df[f'FM_1.{i}_A'] == 1].drop(columns = gold_ranges + ['MEAS_DT'])  # Признаки

        y = result_df[result_df[f'FM_1.{i}_A'] == 1][gold_ranges] # Эталонные диапазоны

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
        
        model.fit(X_train, y_train)

        X = df.drop(columns = gold_ranges + ['MEAS_DT'])
        y_pred = model.predict(X)

        y_pred_df = pd.DataFrame(y_pred, columns=gold_ranges)
        BIG_result_df = pd.concat([BIG_result_df, y_pred_df], axis=1)

In [8]:
#2

In [9]:
for i in range(1, 3):

        gold_ranges = [f'Cu_2.{i}T_min', f'Cu_2.{i}T_max']

        X = result_df[result_df[f'FM_2.{i}_A'] == 1].drop(columns = gold_ranges + ['MEAS_DT'])  # Признаки

        y = result_df[result_df[f'FM_2.{i}_A'] == 1][gold_ranges] # Эталонные диапазоны

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
        
        model.fit(X_train, y_train)

        X = df.drop(columns = gold_ranges + ['MEAS_DT'])
        y_pred = model.predict(X)

        y_pred_df = pd.DataFrame(y_pred, columns=gold_ranges)
        BIG_result_df = pd.concat([BIG_result_df, y_pred_df], axis=1)

In [10]:
#3

In [11]:
for i in range(1, 3):

        gold_ranges = [f'Cu_3.{i}T_min', f'Cu_3.{i}T_max']

        X = result_df[result_df[f'FM_3.{i}_A'] == 1].drop(columns = gold_ranges + ['MEAS_DT'])  # Признаки

        y = result_df[result_df[f'FM_3.{i}_A'] == 1][gold_ranges] # Эталонные диапазоны

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
        
        model.fit(X_train, y_train)

        X = df.drop(columns = gold_ranges + ['MEAS_DT'])
        y_pred = model.predict(X)

        y_pred_df = pd.DataFrame(y_pred, columns=gold_ranges)
        BIG_result_df = pd.concat([BIG_result_df, y_pred_df], axis=1)

In [12]:
#4

In [13]:
for i in range(1, 3):

        gold_ranges = [f'Ni_4.{i}T_min', f'Ni_4.{i}T_max', f'Ni_4.{i}C_min', f'Ni_4.{i}C_max']

        X = result_df[result_df[f'FM_4.{i}_A'] == 1].drop(columns = gold_ranges + ['MEAS_DT'])  # Признаки

        y = result_df[result_df[f'FM_4.{i}_A'] == 1][gold_ranges] # Эталонные диапазоны

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
        
        model.fit(X_train, y_train)

        X = df.drop(columns = gold_ranges + ['MEAS_DT'])
        y_pred = model.predict(X)

        y_pred_df = pd.DataFrame(y_pred, columns=gold_ranges)
        BIG_result_df = pd.concat([BIG_result_df, y_pred_df], axis=1)

In [14]:
#5

In [15]:
for i in range(1, 3):

        gold_ranges = [f'Ni_5.{i}T_min', f'Ni_5.{i}T_max', f'Ni_5.{i}C_min', f'Ni_5.{i}C_max']

        X = result_df[result_df[f'FM_5.{i}_A'] == 1].drop(columns = gold_ranges + ['MEAS_DT'])  # Признаки

        y = result_df[result_df[f'FM_5.{i}_A'] == 1][gold_ranges] # Эталонные диапазоны

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
        
        model.fit(X_train, y_train)

        X = df.drop(columns = gold_ranges + ['MEAS_DT'])
        y_pred = model.predict(X)

        y_pred_df = pd.DataFrame(y_pred, columns=gold_ranges)
        BIG_result_df = pd.concat([BIG_result_df, y_pred_df], axis=1)

In [16]:
#6

In [17]:
for i in range(1, 3):

        gold_ranges = [f'Ni_6.{i}T_min', f'Ni_6.{i}T_max', f'Ni_6.{i}C_min', f'Ni_6.{i}C_max']

        X = result_df[result_df[f'FM_6.{i}_A'] == 1].drop(columns = gold_ranges + ['MEAS_DT'])  # Признаки

        y = result_df[result_df[f'FM_6.{i}_A'] == 1][gold_ranges] # Эталонные диапазоны

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
        
        model.fit(X_train, y_train)

        X = df.drop(columns = gold_ranges + ['MEAS_DT'])
        y_pred = model.predict(X)

        y_pred_df = pd.DataFrame(y_pred, columns=gold_ranges)
        BIG_result_df = pd.concat([BIG_result_df, y_pred_df], axis=1)

In [18]:
BIG_result_df

,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Ni_1.2C_min,Ni_1.2C_max,Cu_1.2C_min,Cu_1.2C_max,Cu_2.1T_min,Cu_2.1T_max,...,Ni_5.2C_min,Ni_5.2C_max,Ni_6.1T_min,Ni_6.1T_max,Ni_6.1C_min,Ni_6.1C_max,Ni_6.2T_min,Ni_6.2T_max,Ni_6.2C_min,Ni_6.2C_max
0,2.628,3.150,4.367,4.957,2.752,3.206,4.597,5.171,0.3108,0.3854,...,6.184,6.867,1.2350,1.4185,8.862,9.070,1.2127,1.4250,8.822,9.154
1,2.701,3.205,4.505,4.970,2.699,3.204,4.603,5.052,0.3246,0.3896,...,6.222,6.563,1.2975,1.3995,9.112,9.322,1.2872,1.4230,8.885,9.441
2,2.719,3.205,4.513,4.993,2.694,3.204,4.598,5.050,0.3254,0.3961,...,6.184,6.555,1.2930,1.3960,9.106,9.313,1.2852,1.4195,8.904,9.448
3,2.723,3.205,4.524,4.997,2.691,3.202,4.588,5.034,0.3239,0.3876,...,6.193,6.552,1.2960,1.3975,9.096,9.316,1.2812,1.3995,8.911,9.439
4,2.730,3.204,4.535,4.982,2.690,3.202,4.578,5.051,0.3231,0.3851,...,6.195,6.554,1.2915,1.4000,9.108,9.324,1.2852,1.4110,8.910,9.440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30331,2.504,3.289,4.468,4.933,2.670,3.228,4.578,5.165,0.2904,0.3931,...,6.175,6.824,1.2870,1.3995,9.116,9.343,1.2715,1.3945,9.009,9.421
30332,2.506,3.289,4.469,4.936,2.669,3.226,4.587,5.172,0.2883,0.3984,...,6.171,6.824,1.2935,1.3980,9.116,9.353,1.2700,1.3945,9.032,9.426
30333,2.506,3.289,4.470,4.937,2.672,3.226,4.592,5.165,0.2907,0.3958,...,6.167,6.823,1.2930,1.3990,9.129,9.365,1.2686,1.3925,9.037,9.440
30334,2.504,3.289,4.467,4.942,2.675,3.226,4.591,5.183,0.2875,0.3972,...,6.213,6.843,1.2980,1.3875,9.203,9.437,1.2757,1.3920,9.183,9.455


In [19]:
# Функция для агрегации значений в блоки по 8 строк и замены значений на среднее
def aggregate_blocks(BIG_result_df, block_size=8):
    # Создаем копию исходного DataFrame, чтобы не изменять его напрямую
    aggregated_df = BIG_result_df.copy()
    
    # Для каждого столбца в DataFrame вычисляем среднее по блокам
    for col in aggregated_df.columns:
        # Группируем по блокам и заменяем все строки в блоке на среднее
        aggregated_df[col] = aggregated_df[col].groupby(aggregated_df.index // block_size).transform('mean')
    
    return aggregated_df

df_aggregated = aggregate_blocks(BIG_result_df, block_size=8)

In [20]:
increment_map = {
    'Ni_1.1C_min': 0.1,
    'Ni_1.1C_max': 0.1,
    'Cu_1.1C_min': 0.1,
    'Cu_1.1C_max': 0.1,
    'Cu_2.1T_min': 0.01,
    'Cu_2.1T_max': 0.01,
    'Cu_3.1T_min': 0.05,
    'Cu_3.1T_max': 0.05,
    'Ni_4.1T_min': 0.01,
    'Ni_4.1T_max': 0.01,
    'Ni_4.1C_min': 0.05,
    'Ni_4.1C_max': 0.05,
    'Ni_5.1T_min': 0.01,
    'Ni_5.1T_max': 0.01,
    'Ni_5.1C_min': 0.05,
    'Ni_5.1C_max': 0.05,
    'Ni_6.1T_min': 0.01,
    'Ni_6.1T_max': 0.01,
    'Ni_6.1C_min': 0.05,
    'Ni_6.1C_max': 0.05,
}

y_pred = df_aggregated

# Функция округления значений предсказанных диапазонов
def apply_increments(predictions, increment_map):
    # Для каждого признака применяем соответствующее приращение
    for column in predictions.columns:
        increment = increment_map.get(column, 0.1)  # Если нет в map, по умолчанию 0.1
        predictions[column] = np.round(predictions[column] / increment) * increment
    return predictions

y_pred_df = pd.DataFrame(y_pred, columns=[
    'Ni_1.1C_min', 'Ni_1.1C_max', 'Cu_1.1C_min', 'Cu_1.1C_max',
    'Cu_2.1T_min', 'Cu_2.1T_max',
    'Cu_3.1T_min', 'Cu_3.1T_max',
    'Ni_4.1T_min', 'Ni_4.1T_max', 'Ni_4.1C_min', 'Ni_4.1C_max',
    'Ni_5.1T_min', 'Ni_5.1T_max', 'Ni_5.1C_min', 'Ni_5.1C_max',
    'Ni_6.1T_min', 'Ni_6.1T_max', 'Ni_6.1C_min', 'Ni_6.1C_max',
])

y_pred_rounded = apply_increments(y_pred_df, increment_map)

In [21]:
BIG_result_df = pd.concat([df['MEAS_DT'], y_pred_rounded], axis=1)
BIG_result_df

,MEAS_DT,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Cu_2.1T_min,Cu_2.1T_max,Cu_3.1T_min,Cu_3.1T_max,Ni_4.1T_min,...,Ni_4.1C_min,Ni_4.1C_max,Ni_5.1T_min,Ni_5.1T_max,Ni_5.1C_min,Ni_5.1C_max,Ni_6.1T_min,Ni_6.1T_max,Ni_6.1C_min,Ni_6.1C_max
0,2024-01-01 00:00:00,2.7,3.2,4.5,5.0,0.32,0.39,0.75,1.75,0.66,...,4.15,4.55,0.56,0.71,6.10,6.65,1.29,1.40,9.10,9.3
1,2024-01-01 00:15:00,2.7,3.2,4.5,5.0,0.32,0.39,0.75,1.75,0.66,...,4.15,4.55,0.56,0.71,6.10,6.65,1.29,1.40,9.10,9.3
2,2024-01-01 00:30:00,2.7,3.2,4.5,5.0,0.32,0.39,0.75,1.75,0.66,...,4.15,4.55,0.56,0.71,6.10,6.65,1.29,1.40,9.10,9.3
3,2024-01-01 00:45:00,2.7,3.2,4.5,5.0,0.32,0.39,0.75,1.75,0.66,...,4.15,4.55,0.56,0.71,6.10,6.65,1.29,1.40,9.10,9.3
4,2024-01-01 01:00:00,2.7,3.2,4.5,5.0,0.32,0.39,0.75,1.75,0.66,...,4.15,4.55,0.56,0.71,6.10,6.65,1.29,1.40,9.10,9.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30331,2024-11-11 22:45:00,2.5,3.3,4.5,4.9,0.29,0.40,0.75,1.75,0.68,...,4.00,5.20,0.57,0.70,6.05,6.80,1.29,1.39,9.15,9.4
30332,2024-11-11 23:00:00,2.5,3.3,4.5,4.9,0.29,0.40,0.75,1.75,0.68,...,4.00,5.20,0.57,0.70,6.05,6.80,1.29,1.39,9.15,9.4
30333,2024-11-11 23:15:00,2.5,3.3,4.5,4.9,0.29,0.40,0.75,1.75,0.68,...,4.00,5.20,0.57,0.70,6.05,6.80,1.29,1.39,9.15,9.4
30334,2024-11-11 23:30:00,2.5,3.3,4.5,4.9,0.29,0.40,0.75,1.75,0.68,...,4.00,5.20,0.57,0.70,6.05,6.80,1.29,1.39,9.15,9.4
